In [1]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

c:\Users\baigs\AppData\Local\miniconda3\envs\DSML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- Load FAISS index and metadata ---
INDEX_PATH = "data/laws_index.faiss"
META_PATH = "data/laws_metadata.json"

print("🔹 Loading index and metadata...")
index = faiss.read_index(INDEX_PATH)
with open(META_PATH, "r", encoding="utf-8") as f:
    metadata = json.load(f)

# --- Load same embedding model used for dataset ---
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# --- Retrieval Function ---
def retrieve_laws(query: str, top_k: int = 3):
    """
    Given a user query, returns the top_k most relevant law sections
    from the dataset using semantic search (FAISS).
    """
    # Convert query to embedding
    query_vector = model.encode([query], convert_to_numpy=True)
    
    # Search in FAISS index
    distances, indices = index.search(query_vector, top_k)
    
    results = []
    keys = list(metadata.keys())

    for idx, dist in zip(indices[0], distances[0]):
        if idx < len(keys):
            entry = metadata[keys[idx]]
            results.append({
                "title": entry.get("title"),
                "source": entry.get("source"),
                "text": entry.get("text")[:1000],  # Limit preview length
                "distance": float(dist)
            })
    
    return results


# --- Quick test ---
if __name__ == "__main__":
    query = input("Enter a legal question: ")
    results = retrieve_laws(query, top_k=3)
    for i, r in enumerate(results, 1):
        print(f"\n🔹 Result {i}")
        print("Title:", r["title"])
        print("Source:", r["source"])
        print("Snippet:", r["text"][:300], "...")
        print("Distance:", r["distance"])


🔹 Loading index and metadata...

🔹 Result 1
Title: Section 57
Source: Code of Criminal Procedure, 1898
Snippet: Section 57. If there is no sufficient reason to believe that he has committed any offence; he shall be at once released. 60. Person arrested to be taken before Magistrate or officer incharge of policestation: A police officer making an arrest without warrant shall, without unnecessary delay and subj ...
Distance: 0.8336917161941528

🔹 Result 2
Title: Section 565
Source: Code of Criminal Procedure, 1898
Snippet: Section 565, sub-section (3); ninthly, any, person for whose arrest a requisition has been received from another police officer, provided that the requisition specifies the person to be arrested and the offence or other cause for which the arrest is to be made and it appears therefrom that the perso ...
Distance: 0.9507039785385132

🔹 Result 3
Title: Section 494
Source: Code of Criminal Procedure, 1898
Snippet: Section 494, and the provisions of that section, shall app